In [ ]:
import sqlite3
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Mean Pooling - Take attention mask into account for correct averaging


def mean_pooling(model_output, attention_mask):
    # First element of model_output contains all token embeddings
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(
    'sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Define the English question
query_sentence = 'What are names of films?'

# Connect to database and fetch table names and column names
# Update the path to the database file
conn = sqlite3.connect('/content/Db-IMDB.db')
cursor = conn.cursor()

table_names = [table_info[0] for table_info in cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='table';").fetchall()]
column_names = []
for table_name in table_names:
    cursor.execute(f"PRAGMA table_info({table_name});")
    column_names.extend([column_info[1] for column_info in cursor.fetchall()])

# Tokenize query sentence, table names, and column names
query_sentence_encoded = tokenizer(
    [query_sentence], padding=True, truncation=True, return_tensors='pt')
table_names_encoded = tokenizer(
    table_names, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings for query sentence, table names, and column names
with torch.no_grad():
    query_sentence_output = model(**query_sentence_encoded)
    table_names_output = model(**table_names_encoded)

# Perform pooling for query sentence, table names, and column names
query_sentence_embedding = mean_pooling(
    query_sentence_output, query_sentence_encoded['attention_mask'])
table_names_embeddings = mean_pooling(
    table_names_output, table_names_encoded['attention_mask'])

# Normalize embeddings for query sentence, table names, and column names
query_sentence_embedding = F.normalize(query_sentence_embedding, p=2, dim=1)
table_names_embeddings = F.normalize(table_names_embeddings, p=2, dim=1)

# Find the most similar table names and column names by computing the cosine similarity between the query sentence embedding and the table names and column names embeddings
cosine_similarities_tables = torch.nn.functional.cosine_similarity(
    query_sentence_embedding, table_names_embeddings, dim=1)
most_similar_table_names_indices = cosine_similarities_tables.argsort(
    descending=True)
most_similar_table_names = [table_names[i]
                            for i in most_similar_table_names_indices]

# Print the most similar table names with there cosine similarity scores in descending order
for i in range(len(most_similar_table_names)):
    print(
        f"Table name: {most_similar_table_names[i]}, cosine similarity score: {cosine_similarities_tables[most_similar_table_names_indices[i]]}")

# Find the index of the highest matching table name by finding the maximum value in the list of cosine similarities for the table names
max_similarity_table_index = cosine_similarities_tables.argmax()

# Get the highest matching table name by using the index obtained above
highest_matching_table_name = table_names[max_similarity_table_index]

# Find the column names of the highest matching table by querying the database
cursor.execute(f"PRAGMA table_info({highest_matching_table_name});")
highest_matching_table_column_names = [
    column_info[1] for column_info in cursor.fetchall()]

# Tokenize the column names of the highest matching table
highest_matching_table_column_names_encoded = tokenizer(
    highest_matching_table_column_names, padding=True, truncation=True, return_tensors='pt')

# Compute the token embeddings for the column names of the highest matching table
with torch.no_grad():
    highest_matching_table_column_names_output = model(
        **highest_matching_table_column_names_encoded)

# Perform mean pooling on the output of the language model for the column names of the highest matching table
highest_matching_table_column_names_embeddings = mean_pooling(
    highest_matching_table_column_names_output, highest_matching_table_column_names_encoded['attention_mask'])

# Normalize the embeddings for the column names of the highest matching table
highest_matching_table_column_names_embeddings = F.normalize(
    highest_matching_table_column_names_embeddings, p=2, dim=1)

# Compute the cosine similarity between the query sentence embedding and the column names embeddings of the highest matching table
cosine_similarities_highest_matching_table_columns = torch.nn.functional.cosine_similarity(
    query_sentence_embedding, highest_matching_table_column_names_embeddings, dim=1)

# Find the most similar column name in the highest matching table by sorting the cosine similarities in descending order
most_similar_highest_matching_table_column_name_index = cosine_similarities_highest_matching_table_columns.argmax()
most_similar_highest_matching_table_column_name = highest_matching_table_column_names[
    most_similar_highest_matching_table_column_name_index]


# Generate an SQL SELECT query based on the most similar table names and column names
query = f"SELECT {most_similar_highest_matching_table_column_name} FROM {most_similar_table_names[0]}"

# Iterate through the list of possible queries and execute each one

# Print the generated SQL query
print(query)

try:
    cursor.execute(query)
    results = cursor.fetchall()
    print(f'Query: {query}')
    print(f'Results: {results}')
except Exception as e:
    print(f'Error: {e}')


# close the connection to the database
conn.close()
